In [1]:
import pandas as pd
import numpy as np
import os
import enum
import time
import matplotlib.pyplot as plt
import json
import seaborn as sns
import ignite

from tqdm.auto import tqdm, trange

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchio as tio
import monai

import nilearn as nl
import nibabel as nib
import nilearn.plotting as nlplt
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.metrics import precision_recall_curve

from matplotlib.colors import ListedColormap

from datetime import datetime
from functools import partialmethod

# from google.colab import drive

torch.manual_seed(7)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

torch.set_grad_enabled(False)

In [1]:
from pytorch0 import DiffusionMRIDataModule, MONAIDiffusionMRIDataModule

batch_size = 8
num_workers = 8
multiplier = 10

test_subjs_idx = [2, 16, 13, 12]
val_subjs_idx = [0, 7, 10, 14]

dMRI = DiffusionMRIDataModule(val_subjs_idx, test_subjs_idx, "../../study", batch_size, num_workers, multiplier)
dMRI.setup(stage="fit")
train_dataloader = dMRI.train_dataloader()

In [2]:
%time next(iter(train_dataloader))

CPU times: user 288 ms, sys: 441 ms, total: 729 ms
Wall time: 10.3 s


{'id': ['VNSLC_12',
  'VNSLC_12',
  'VNSLC_04',
  'VNSLC_23',
  'VNSLC_18',
  'VNSLC_14',
  'VNSLC_23',
  'VNSLC_04'],
 'resp': tensor([1., 1., 1., 0., 0., 1., 0., 1.], dtype=torch.float64),
 'wfvf': {'data': tensor([[[[[-1.4504, -1.4340, -1.4367,  ..., -1.4123, -1.4081, -1.3712],
             [-1.3955, -1.4296, -1.3494,  ..., -1.3937, -1.4336, -1.3978],
             [-1.5008, -1.4744, -1.3643,  ..., -1.4496, -1.4038, -1.4674],
             ...,
             [-1.4545, -1.3495, -1.4412,  ..., -1.4266, -1.4824, -1.3992],
             [-1.4325, -1.3631, -1.3670,  ..., -1.4361, -1.4622, -1.4359],
             [-1.3680, -1.4405, -1.4058,  ..., -1.4565, -1.3696, -1.4052]],
  
            [[-1.3819, -1.4331, -1.3965,  ..., -1.3587, -1.2828, -1.4233],
             [-1.4628, -1.4166, -1.4034,  ..., -1.4112, -1.4121, -1.3706],
             [-1.4047, -1.4160, -1.4401,  ..., -1.4458, -1.4490, -1.4560],
             ...,
             [-1.4286, -1.4422, -1.3868,  ..., -1.4549, -1.3566, -1.4912],
   